1. 게임 설정 입력 <br>
   ↓ <br> <br>
2. GameDifficultyCalculator 초기화 <br>
   ├── 모든 실력 수준 (0.1~0.9)에 대해 <br>
   ├── 모든 단계 (1~7)별로 <br>
   └── 기존 함수들로 성능 미리 계산 <br> 
   ↓ <br> <br>
3. 실제 데이터 입력 → UserPopulationEstimator <br>
   ├── 각 단계별 실패율 분석 <br>
   ├── 미리 계산된 프로필과 비교 <br>
   └── 유저 실력 분포 추정 <br>
   ↓ <br> <br>
4. AdaptiveMonteCarlo 시뮬레이션 <br>
   ├── 추정된 실력 분포로 유저 샘플링 <br>
   ├── 미리 계산된 성공률로 빠른 시뮬레이션 <br>
   └── 결과 분석 및 난이도 조정 권장 <br>

## 현재 게임 규칙

1단계2x2 / 총 그림 2개- 기본 카드: 2개- 트릭 카드: 0개제한 시간: 5초성공 리워드: 2원실패 리워드: 1원

2단계2x3 / 총 그림 3개- 기본 카드: 3개- 트릭 카드: 0개제한 시간: 7초성공 리워드: 2원

3단계3x4 / 총 그림 6개- 기본 카드: 6개- 트릭 카드: 0개제한 시간: 9초성공 리워드: 2원

4단계4x4 / 총 그림 8개- 기본 카드: 8개- 트릭 카드: 0개제한 시간: 13초성공 리워드: 2원

5단계4x4 / 총 그림 8개- 기본 카드: 6개- 트릭 카드: 2개제한 시간: 15초성공 리워드: 3원

6단계4x5 / 총 그림 10개- 기본 카드: 10개- 트릭 카드: 0개제한 시간: 21초성공 리워드: 3원

7단계4x5 / 총 그림 10개- 기본 카드: 8개- 트릭 카드: 2개제한 시간: 25초성공 리워드: 6원

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import pandas as pd
import scipy.sparse
import scipy.sparse.linalg
import sys
# 맥과 윈도우 os 판별하고, 시각화에서 한글 꺠지지 않도록 설정하는 코드
if sys.platform == 'darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'Malgun Gothic'

import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import scipy.sparse
import scipy.sparse.linalg
from typing import Tuple, Dict, Any, Optional

def remove_last_row_column(sparse_matrix):
    """행렬에서 마지막 행과 열을 제거하는 함수"""
    num_states = sparse_matrix.shape[0]
    row_mask = np.ones(sparse_matrix.shape[0], dtype=bool)
    row_mask[num_states-1] = False
    col_mask = np.ones(sparse_matrix.shape[1], dtype=bool)
    col_mask[num_states-1] = False
    return sparse_matrix[row_mask][:,col_mask]

def _build_matrix_from_adjacency(adjacency_list: Dict, entire_matrix: bool, extra_info: bool):
    """card_matching.ipynb와 동일한 행렬 구성"""
    num_states = len(adjacency_list)
    markov_chain = scipy.sparse.lil_matrix((num_states, num_states))

    # Build enumeration for states - card_matching.ipynb와 동일
    sorted_keys = sorted(list(adjacency_list.keys()), key=lambda pair: -sum(pair))
    keys_to_index = {key: index for index, key in enumerate(sorted_keys)}

    for state, neighbors in adjacency_list.items():
        adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 1e-10}
        for destination, probability in neighbors.items():
            markov_chain[(keys_to_index[state], keys_to_index[destination])] = probability
    
    if not entire_matrix:  # Get block matrix for fundamental matrix calculation
        to_return = remove_last_row_column(markov_chain)
    else:
        to_return = scipy.sparse.csr_matrix(markov_chain)

    if extra_info:
        return to_return, {"adjacency_list": adjacency_list, "keys_to_index": keys_to_index, "sorted_keys": sorted_keys}
    return to_return

def get_optimal_baseline_markov_chain(number_of_pairs: int, entire_matrix: bool = False, extra_info: bool = False):
    """card_matching.ipynb와 동일한 완벽한 기억력 베이스라인"""
    queue = [(0, 2 * number_of_pairs)]
    adjacency_list = {}

    while queue:
        curr_state = queue.pop()
        alpha, beta = curr_state
    
        if alpha < 0 or beta < 0:
            continue
  
        next_states = {}
        # card_matching.ipynb의 정확한 로직
        if alpha > beta or (alpha + beta) % 2 == 1:  # Know more knowns than unknowns or there is a duplicate in the flipped cards
            next_states[(alpha - 1, beta)] = 1  # 완전 기억 케이스
        elif alpha == 0 and beta == 0:
            next_states[(0, 0)] = 1
        elif alpha == beta:
            next_states[(max(0, alpha - 1), beta - 1)] = 1
        else: 
            # 완전 기억 케이스 - card_matching.ipynb와 동일
            next_states[(alpha - 1, beta - 1)] = alpha / beta 
            next_states[(alpha + 2, beta - 2)] = (1 - alpha/beta) * (1 - (alpha+1)/(beta-1))
            next_states[(alpha + 1, beta - 2)] = (1 - alpha/beta) * (alpha / (beta - 1))
            next_states[(alpha, beta - 2)] = (1 - alpha/beta) * 1 / (beta - 1)

        # exclude calculating transitions for states with 0 probability to reach
        queue.extend(state for state, probability in next_states.items() 
                    if state not in adjacency_list and probability > 0)
        adjacency_list[curr_state] = next_states

    # Clear out not possible states
    for state, neighbors in adjacency_list.items():
        adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 0}

    return _build_matrix_from_adjacency(adjacency_list, entire_matrix, extra_info)

def get_imperfect_memory_markov_chain(number_of_pairs: int, memory_decay_factor: float = 0.1, 
                                     entire_matrix: bool = False, extra_info: bool = False):
    """기억력 저하로 인한 성능 감소 모델 - 턴 수가 증가하도록 설계"""
    queue = [(0, 2 * number_of_pairs)]
    adjacency_list = {}

    while queue:
        curr_state = queue.pop()
        alpha, beta = curr_state
    
        if alpha < 0 or beta < 0:
            continue
  
        next_states = {}
        
        # 게임 종료 조건들
        if alpha == 0 and beta == 0:
            next_states[(0, 0)] = 1
        elif beta == 0:
            next_states[(0, 0)] = 1
        elif beta == 1:
            next_states[(0, 0)] = 1
        elif (alpha + beta) % 2 == 1:
            next_states[(max(0, alpha - 1), beta)] = 1
        else:
            # 기억력 저하 효과 적용
            base_success_prob = alpha / beta if alpha <= beta else 0.9
            
            # memory_decay_factor가 클수록 성능 저하 (더 많은 턴 필요)
            degraded_success_prob = base_success_prob * (1 - memory_decay_factor)
            
            # 매칭 성공 (기억한 정보로 올바른 선택)
            success_prob = min(degraded_success_prob, 0.95)  # 최대 95%로 제한
            next_states[(max(0, alpha - 1), beta - 2)] = success_prob
            
            # 실패 케이스들 (기억력 저하로 인한 비효율적 선택)
            fail_prob = 1 - success_prob
            
            if fail_prob > 0:
                # 완전 실패: 새로운 정보만 얻고 매칭 실패
                next_states[(min(alpha + 2, beta), beta - 2)] = fail_prob * 0.6
                
                # 부분 실패: 일부 정보만 기억하고 매칭 실패  
                next_states[(min(alpha + 1, beta), beta - 2)] = fail_prob * 0.3
                
                # 정보 손실: 기존 정보도 일부 잃음
                next_states[(max(0, alpha - 1), beta)] = fail_prob * 0.1

        queue.extend(state for state, probability in next_states.items() 
                    if state not in adjacency_list and probability > 0)
        adjacency_list[curr_state] = next_states

    # 0 확률 상태 제거
    for state, neighbors in adjacency_list.items():
        adjacency_list[state] = {k: v for k, v in neighbors.items() if v > 0}

    return _build_matrix_from_adjacency(adjacency_list, entire_matrix, extra_info)

def get_case_statistics(markov_matrix, n_pairs: int) -> np.ndarray:
    """card_matching.ipynb와 동일한 분포 계산"""
    starting_point = np.zeros(markov_matrix.shape[0])
    starting_point[0] = 1

    first_chance_to_win = markov_matrix.T ** n_pairs
    prob_vector = first_chance_to_win @ starting_point
    chance_to_finish_with_failures = [prob_vector[-1]]

    # card_matching.ipynb와 동일한 범위
    for number_of_failures in range(n_pairs):
        next_prob_vector = markov_matrix.T @ prob_vector
        chance_to_finish_with_failures.append(next_prob_vector[-1] - prob_vector[-1])
        prob_vector = next_prob_vector
    
    return np.array(chance_to_finish_with_failures)

def calculate_turn_penalty(skill_level: float, baseline_turns: float) -> float:
    """실력 수준에 따른 턴 수 증가 계산"""
    
    # skill_level: 1.0 (완벽) -> 0.0 (최악)
    # 베이스라인 대비 턴 수 증가율
    
    if skill_level >= 0.9:
        penalty_multiplier = 1.0  # 베이스라인 수준
    elif skill_level >= 0.7:
        penalty_multiplier = 1.3  # 30% 증가
    elif skill_level >= 0.5:
        penalty_multiplier = 1.8  # 80% 증가
    elif skill_level >= 0.3:
        penalty_multiplier = 2.5  # 150% 증가
    elif skill_level >= 0.1:
        penalty_multiplier = 3.5  # 250% 증가
    else:
        penalty_multiplier = 5.0  # 400% 증가
    
    return baseline_turns * penalty_multiplier

def skill_level_to_memory_decay(skill_level: float) -> float:
    """실력 수준을 memory_decay_factor로 변환"""
    # 높은 실력 = 낮은 decay, 낮은 실력 = 높은 decay
    return max(0.0, min(0.8, (1.0 - skill_level) * 0.8))

def get_user_performance_metrics(n_pairs: int, skill_level: float) -> Dict[str, float]:
    """
    실력 수준에 따른 성과 지표 계산 - 베이스라인 대비 턴 수 증가 중심
    """
    # 베이스라인 계산
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    baseline_completion_rate = sum(baseline_stats)
    
    # 사용자 성능 계산
    if skill_level >= 0.95:
        # 거의 완벽한 사용자는 베이스라인과 동일
        user_stats = baseline_stats
        user_expected_failures = baseline_expected_failures
        user_completion_rate = baseline_completion_rate
    else:
        # 기억력 저하가 있는 사용자
        memory_decay = skill_level_to_memory_decay(skill_level)
        user_matrix = get_imperfect_memory_markov_chain(n_pairs, memory_decay, entire_matrix=True)
        user_stats = get_case_statistics(user_matrix, n_pairs)
        user_completion_rate = sum(user_stats)
        
        if user_completion_rate > 0:
            user_expected_failures = sum(i * prob for i, prob in enumerate(user_stats)) / user_completion_rate
        else:
            user_expected_failures = float('inf')
    
    # 실제 턴 수 (패널티 적용)
    actual_turns = calculate_turn_penalty(skill_level, baseline_turns)
    
    # 성능 지표
    turn_increase_ratio = actual_turns / baseline_turns if baseline_turns > 0 else float('inf')
    efficiency_score = baseline_turns / actual_turns if actual_turns > 0 else 0.0
    
    return {
        'skill_level': skill_level,
        'baseline_turns': baseline_turns,
        'actual_turns': actual_turns,
        'turn_increase_ratio': turn_increase_ratio,
        'efficiency_score': efficiency_score,
        'expected_failures': user_expected_failures,
        'completion_rate': user_completion_rate,
        'baseline_completion_rate': baseline_completion_rate
    }

def monte_carlo_user_simulation(n_pairs: int, user_skill_distribution: list, 
                               skill_levels: list = [0.1, 0.3, 0.5, 0.7, 0.9],
                               num_simulations: int = 10000) -> dict:
    """유저 실력 분포를 기반으로 한 몬테카를로 시뮬레이션"""
    
    # 베이스라인 계산
    baseline_matrix = get_optimal_baseline_markov_chain(n_pairs, entire_matrix=True)
    baseline_stats = get_case_statistics(baseline_matrix, n_pairs)
    baseline_expected_failures = sum(i * prob for i, prob in enumerate(baseline_stats))
    baseline_turns = baseline_expected_failures + n_pairs
    
    results = []
    
    for simulation in range(num_simulations):
        # 유저 실력 샘플링
        user_skill = np.random.choice(skill_levels, p=user_skill_distribution)
        
        # 해당 실력에 따른 실제 턴 수 계산
        actual_turns = calculate_turn_penalty(user_skill, baseline_turns)
        
        # 게임 경제성 계산
        ad_revenue_per_turn = 0.05  # 턴당 광고 수익
        completion_reward = 3.0
        
        ad_revenue = actual_turns * ad_revenue_per_turn
        net_profit = ad_revenue - completion_reward
        
        results.append({
            'user_skill': user_skill,
            'baseline_turns': baseline_turns,
            'actual_turns': actual_turns,
            'turn_increase': actual_turns - baseline_turns,
            'turn_increase_ratio': actual_turns / baseline_turns,
            'ad_revenue': ad_revenue,
            'net_profit': net_profit
        })
    
    # 통계 계산
    avg_turns = np.mean([r['actual_turns'] for r in results])
    avg_turn_increase = np.mean([r['turn_increase'] for r in results])
    avg_profit = np.mean([r['net_profit'] for r in results])
    
    return {
        'simulations': results,
        'baseline_turns': baseline_turns,
        'avg_actual_turns': avg_turns,
        'avg_turn_increase': avg_turn_increase,
        'avg_turn_increase_ratio': avg_turns / baseline_turns,
        'avg_profit': avg_profit,
        'total_simulations': num_simulations
    }



In [ ]:
def example_usage():
    """사용 예시"""
    n_pairs = 8
    
    print("=== 베이스라인 vs 다양한 실력 수준 비교 ===")
    skill_levels = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]
    
    for skill in skill_levels:
        metrics = get_user_performance_metrics(n_pairs, skill)
        print(f"실력 {skill:.1f}: 베이스라인 {metrics['baseline_turns']:.2f}턴 → "
              f"실제 {metrics['actual_turns']:.2f}턴 "
              f"(+{metrics['turn_increase_ratio']:.1%})")
    
    print("\n=== 몬테카를로 시뮬레이션 ===")
    # 실제적인 유저 분포 (초보자가 많음)
    user_distribution = [0.4, 0.3, 0.2, 0.08, 0.02]  # [0.1, 0.3, 0.5, 0.7, 0.9]
    
    results = monte_carlo_user_simulation(
        n_pairs=n_pairs,
        user_skill_distribution=user_distribution,
        num_simulations=5000
    )
    
    print(f"베이스라인 평균 턴 수: {results['baseline_turns']:.2f}")
    print(f"실제 평균 턴 수: {results['avg_actual_turns']:.2f}")
    print(f"평균 턴 수 증가: +{results['avg_turn_increase']:.2f}턴 "
          f"(+{(results['avg_turn_increase_ratio']-1)*100:.1f}%)")
    print(f"평균 순이익: ${results['avg_profit']:.3f}")
    
    return results

if __name__ == "__main__":
    example_usage()

=== 베이스라인 vs 다양한 실력 수준 비교 ===
실력 1.0: 베이스라인 12.39턴 → 실제 12.39턴 (+100.0%)
실력 0.8: 베이스라인 12.39턴 → 실제 16.11턴 (+130.0%)
실력 0.6: 베이스라인 12.39턴 → 실제 22.31턴 (+180.0%)
실력 0.4: 베이스라인 12.39턴 → 실제 30.98턴 (+250.0%)
실력 0.2: 베이스라인 12.39턴 → 실제 43.38턴 (+350.0%)
실력 0.0: 베이스라인 12.39턴 → 실제 61.96턴 (+500.0%)

=== 몬테카를로 시뮬레이션 ===
베이스라인 평균 턴 수: 12.39
실제 평균 턴 수: 32.52
평균 턴 수 증가: +20.12턴 (+162.4%)
평균 순이익: $-1.374


In [4]:
# 

### 시뮬



In [8]:
# 게임 변수
MAX_RETRIES = 3      # 최대 재시도 횟수
AD_REVENUE = 1.0     # 광고 수익

# 게임 단계 설정
GAME_STAGES = [
    {'pairs': 2, 'time_limit': 5.0, 'reward': 2},    # 1단계: 2x2, 2페어
    {'pairs': 3, 'time_limit': 7.0, 'reward': 2},    # 2단계: 2x3, 3페어  
    {'pairs': 6, 'time_limit': 9.0, 'reward': 2},    # 3단계: 3x4, 6페어
    {'pairs': 8, 'time_limit': 13.0, 'reward': 2},   # 4단계: 4x4, 8페어
    {'pairs': 8, 'time_limit': 15.0, 'reward': 3},   # 5단계: 4x4, 8페어 (트릭카드)
    {'pairs': 10, 'time_limit': 21.0, 'reward': 3},  # 6단계: 4x5, 10페어
    {'pairs': 10, 'time_limit': 25.0, 'reward': 6}   # 7단계: 4x5, 10페어 (트릭카드)
]
